In [9]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc


In [10]:

df = pd.read_csv('district_forecast_2025_2030_all_fields_linear.csv')
df.columns = [c.strip().replace(' ', '_') for c in df.columns]
gdf = gpd.read_file('india_districts.geojson')

for year in df['Year'].unique():
    merged = gdf.merge(df[df['Year'] == year], left_on='NAME_2', right_on='District', how='left')
    fig = px.choropleth_map(
        merged,
        geojson=merged.__geo_interface__,
        locations=merged.index,
        color='Risk_Factor',
        color_continuous_scale=['red', 'orange', 'yellow', 'green'],
        hover_data={
            'NAME_2': True,
            'State': True,
            'Risk_Factor': ':.3f',
            'Sanitation': ':.3f',
            'Literacy': ':.3f',
            'Child_Population_Growth': ':.3f',
            'Year': True
        },
        map_style="carto-positron",
        center={"lat": 22.9734, "lon": 78.6569},
        zoom=4.2,
        opacity=0.7,
        title=f"District-Level Risk Forecast ({year})"
    )
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0}, height=650)
    fig.write_html(f'district_risk_choropleth_{year}.html')


In [11]:
import plotly.graph_objects as go

metrics = ['Risk_Factor', 'Sanitation', 'Literacy', 'Child_Population_Growth']
selected_districts = ['Mumbai', 'Bengaluru', 'Chennai']  # Change as needed
year = 2030

compare_df = df[(df['Year'] == year) & (df['District'].isin(selected_districts))]

# Horizontal bar chart
fig_bar = px.bar(
    compare_df.melt(id_vars=['District'], value_vars=metrics),
    x='value', y='District', color='variable',
    orientation='h', barmode='group',
    labels={'value': 'Score', 'variable': 'Metric'},
    title=f"District Comparison ({year})"
)
fig_bar.write_html('district_comparison_bar_2030.html')

# Radar chart
fig_radar = go.Figure()
for _, row in compare_df.iterrows():
    fig_radar.add_trace(go.Scatterpolar(
        r=[row[m] for m in metrics],
        theta=metrics,
        fill='toself',
        name=row['District']
    ))
fig_radar.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=True,
    title=f"District Radar Comparison ({year})"
)
fig_radar.write_html('district_comparison_radar_2030.html')


In [12]:
# Example: Filter for high literacy and moderate risk
filtered = df[
    (df['Year'] == 2030) &
    (df['Literacy'] > 0.8) &
    (df['Sanitation'] > 0.7) &
    (df['Risk_Factor'] < 0.5)
]
# Now plot as above using this filtered DataFrame
import plotly.graph_objects as go

metrics = ['Risk_Factor', 'Sanitation', 'Literacy', 'Child_Population_Growth']
selected_districts = ['Mumbai', 'Bengaluru', 'Chennai']  # Change as needed
year = 2030

compare_df = df[(df['Year'] == year) & (df['District'].isin(selected_districts))]

# Horizontal bar chart
fig_bar = px.bar(
    compare_df.melt(id_vars=['District'], value_vars=metrics),
    x='value', y='District', color='variable',
    orientation='h', barmode='group',
    labels={'value': 'Score', 'variable': 'Metric'},
    title=f"District Comparison ({year})"
)
fig_bar.write_html('district_comparison_bar_2030.html')

# Radar chart
fig_radar = go.Figure()
for _, row in compare_df.iterrows():
    fig_radar.add_trace(go.Scatterpolar(
        r=[row[m] for m in metrics],
        theta=metrics,
        fill='toself',
        name=row['District']
    ))
fig_radar.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=True,
    title=f"District Radar Comparison ({year})"
)
fig_radar.write_html('district_comparison_radar_2030.html')
